In [1]:
#This is code for Dash application for the capstone project.

#here is first few rows of spacex_launch_dash.csv. First row is header
#very first entry, which is skipped, is index
#,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
#0,1,CCAFS LC-40,0,0,F9 v1.0  B0003,v1.0
#1,2,CCAFS LC-40,0,0,F9 v1.0  B0004,v1.0
#2,3,CCAFS LC-40,0,525,F9 v1.0  B0005,v1.0
#3,4,CCAFS LC-40,0,500,F9 v1.0  B0006,v1.0


# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # A dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', 
                                options=[{'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                ],
                                value='ALL',
                                placeholder='All Sites',
                                searchable = True
                                ),
                                html.Br(),

                                # A pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # A slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=10,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000: '5000',
                                                       7500: '7500',
                                                       10000: '10000'},
                                                value=[0,10000]),

                                # A scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# A callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                    Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        filtered_df = spacex_df.groupby('Launch Site')['class'].sum().reset_index()
        fig = px.pie(filtered_df,
        names='Launch Site',
        values='class',
        title='Total Number of Launch Successes by Launch Site'
        )
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        total_launches_df = filtered_df.groupby('Launch Site')['class'].count().reset_index()
        total_successes_df = filtered_df.groupby('Launch Site')['class'].sum().reset_index()
        total_launches = int(total_launches_df.iloc[0,1])
        total_successes = int(total_successes_df.iloc[0,1])
        fig = px.pie(filtered_df,
        names=['Failure', 'Success'],
        values=[total_launches - total_successes, total_successes],
        title='Ratio between launch success and failure for the launch site ' + entered_site
        )
        return fig

# A callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")
              ])
def get_payload_pie_chart(entered_site, entered_range):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df,
                        x='Payload Mass (kg)', 
                        y='class', 
                        color="Booster Version Category",
                        title="Payload Mass and Launch Success/Failure"
                        )
        fig.update_xaxes(range=entered_range)
        return fig
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df,
                        x='Payload Mass (kg)', 
                        y='class', 
                        color="Booster Version Category",
                        title="Payload Mass vs. Lauch Success/Failure for the Launch Site " + entered_site
                        )
        fig.update_xaxes(range=entered_range)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
